# State of the Art Technology - Prototype 1

In [1]:
#Imports
import cv2
import time
import numpy as np

In [3]:
kernel = np.ones((5,5),np.uint8)
vidCap = cv2.VideoCapture('/Users/raffaelenardi/Documents/GITHUB/State of the Art Technology/Long_road_edit.mov')

# initilize OpenCV - Background Subtractor for KNN and MOG2
BS_KNN = cv2.createBackgroundSubtractorKNN(history=5000, dist2Threshold=2000.0,detectShadows= False)

cars = 0 # Count value
validcars = [] # Container for x and y variables
while vidCap.isOpened():
    ret, frame = vidCap.read() # loops through frames

    # Extract the foreground mask using KNN
    fgMask = BS_KNN.apply(frame)
#     cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel) # clean of contours (add solid fill inside)
    ret,thresh1 = cv2.threshold(fgMask,127,255,cv2.THRESH_BINARY)

# Draw line for counts
    cv2.line(frame,(475,380),(690,355),(0,0,255),2) # Red line to denote area we want to count
    cv2.rectangle(frame,(270,280),(750,600),(255,0,0),2) # Region of interst (ROI)

    # Extract the contours
    conts, _ = cv2.findContours(fgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in conts: # creating bounding rectangles for contours (moving cars)
        x, y, w, h = cv2.boundingRect(c)
        
        # ignore the small contours in size (noise)
        visiblecar = (w > 60) and (h > 60)
        if not visiblecar:
            continue
        
        # remove the distraction on the road; consider only the objects on ROAD
        if x > 270 and x < 750 and y > 280 and y < 600:
            # draw the bounding rectangle for all contours
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
            xMid = int((x + (x+w))/2)
            yMid = int((y + (y+h))/2)
            cv2.circle(frame, (xMid,yMid),5,(0,0,255),10)

            # add all valid cars into List Array
            validcars.append((xMid,yMid))

            for (vX, vY) in validcars:
                if vY == 381 and vX < 680: # detection where number is added to counted cars total
                    cars += 1
                    validcars.remove((vX,vY))
                    
                #debugging purpose
                cv2.putText(frame, 'Y : {}'.format(yMid), (xMid, y-20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)
                cv2.putText(frame, 'X : {}'.format(xMid), (xMid, y-50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)
                cv2.putText(frame, 'w : {}'.format(w), (xMid-150, y-20), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)
                cv2.putText(frame, 'h : {}'.format(h), (xMid-150, y-50), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 2)

                
    # show the thresh and original video
    cv2.imshow('Foreground Mask', fgMask)
    cv2.putText(frame, 'Total cars : {}'.format(cars), (290, 590), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 2)
    cv2.imshow('Original Video', frame)
    
    # wait for any key to be pressedq
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# release video capture
cv2.destroyAllWindows()
vidCap.release()

# Complications

### Technology Issues
- Breaks when two cars are too close together, registers them as a single entity and then fails because its outside ROI
- Issues with using y value as trigger for count because it can be a little jumpy as contour recognition happens
- Sometimes the car does not pass through Y = 380, the trigger value
- Probably need an AI smart enough to pick cars apart

### Human issues
- Not created for the complexity of human movement.
    - Doesn't take into account issues with people stopping in the middle of the round about.
    - People tailgating messes with detection
- Manual set up required for new detection areas. Change of line where count is made and ROI.



## Ideas
- Create a method of tracking point before it crosses the threshold value if point increases past y value and is within x range then add to counter. Check https://medium.com/machine-learning-world/tutorial-making-road-traffic-counting-app-based-on-computer-vision-and-opencv-166937911660 for ideas.

## Thoughts

- Relfect on exploitablity as a concept
- Reflect on what you want to achieve with this. What human values does this satisfy/not satisfy.

- Other Human values
    - Cant provide the human benefit without better technological accuracy
    - Accessibility and empowerment 
        - Ease of use of the ouput
        - Affordability of applying technology (just a camera and a laptop!)
   
    - Collaboration
    
- Google Collaborate - Free GPU
- Keras - Deep learning you can use already trained models (we will do this)